This tutorial is going to show you how to use PyTorch's FSDP implementation. We're going to train two models: (i) a shallow model, and (ii) a deep model with local training. Then, we're going to show how FSDP speeds up our training, and decreases memory.